In [1]:
import cv2
import json
import numpy as np
import pandas as pd
import youtube_dl

In [4]:
vids_df = pd.read_csv('../data/yt8m_indoor_outdoor_labels_id.csv', index_col=0)
vids_df

,labels,target,yt8m_id,yt_id
0,"[88, 184, 420, 514]",indoors,k5a7,LLf2i_EK6RU
1,"[15, 18, 60, 87, 545, 1696]",outdoors,Nda7,bkK-qbwAgJo
2,"[511, 514, 1148, 2277, 2405]",indoors,Dca7,jsFiMne0Eyk
3,[60],outdoors,OEa7,p4Ub6lYdK38
4,"[11, 20, 22, 29, 176, 307, 2561]",indoors,EMa7,KW2WY3GIpt4
...,...,...,...,...
2891,"[11, 262, 370]",outdoors,sa9Y,RF_c2Zp5AhU
2892,"[184, 191, 985]",indoors,jD9Y,3GHSzGIqgTg
2893,"[60, 210, 685]",outdoors,lV9Y,xK2AdjZBNoA
2894,"[60, 2701]",outdoors,F99Y,6w6VXsszoak


### Using Youtube_DL library
Lets see what the info looks like

In [8]:
ydl_options = {}
ydl = youtube_dl.YoutubeDL(ydl_options)
yt_info = ydl.extract_info('https://www.youtube.com/watch?v=LLf2i_EK6RU', download=False)
print(yt_info)

[youtube] LLf2i_EK6RU: Downloading webpage
[youtube] LLf2i_EK6RU: Downloading video info webpage
{'id': 'LLf2i_EK6RU', 'uploader': 'Canal Casa Sul', 'uploader_id': 'ProgramaCasaSul', 'uploader_url': 'http://www.youtube.com/user/ProgramaCasaSul', 'channel_id': 'UCdEC-C0_i1-GJTfrpkEv_qw', 'channel_url': 'http://www.youtube.com/channel/UCdEC-C0_i1-GJTfrpkEv_qw', 'upload_date': '20110829', 'license': None, 'creator': 'Spyro Gyra', 'title': 'IDEIAS PARA ILUMINAR QUARTO E BANHEIRO DE UM JEITO ORIGINAL', 'alt_title': "Not for Nothin'", 'thumbnail': 'https://i.ytimg.com/vi/LLf2i_EK6RU/hqdefault.jpg', 'description': 'Ambientes distintos com um toque em comum: a iluminação. Veja o que foi feito em um quarto de moça e um banheiro público a partir de dicas da GREY HOUSE, na Casacor Paraná 2011 e que continuam atuais', 'categories': ['Howto & Style'], 'tags': ['iluminação', 'decoração de quarto', 'decoração de banheiros', 'greyhouse', 'marcelo lopes', 'rolim de moura', 'projeto luminotécnico', 'ilu

In [9]:
print(yt_info.keys())

dict_keys(['id', 'uploader', 'uploader_id', 'uploader_url', 'channel_id', 'channel_url', 'upload_date', 'license', 'creator', 'title', 'alt_title', 'thumbnail', 'description', 'categories', 'tags', 'subtitles', 'automatic_captions', 'duration', 'age_limit', 'annotations', 'chapters', 'webpage_url', 'view_count', 'like_count', 'dislike_count', 'average_rating', 'formats', 'is_live', 'start_time', 'end_time', 'series', 'season_number', 'episode_number', 'track', 'artist', 'album', 'release_date', 'release_year', 'extractor', 'webpage_url_basename', 'extractor_key', 'playlist', 'playlist_index', 'thumbnails', 'display_id', 'requested_subtitles', 'requested_formats', 'format', 'format_id', 'width', 'height', 'resolution', 'fps', 'vcodec', 'vbr', 'stretched_ratio', 'acodec', 'abr', 'ext'])


In [18]:
formats=yt_info.get('formats', None)
format_notes = [x['format_note'] for x in formats]
print(format_notes)
print(formats[7])

['tiny', 'tiny', 'tiny', 'tiny', '144p', '144p', '240p', '240p', '360p', '360p', '480p', '480p', '360p', '360p']
{'format_id': '133', 'url': 'https://r6---sn-qxoedn7e.googlevideo.com/videoplayback?expire=1573804249&ei=eQTOXbCJNruAir4Pi7eD6Ac&ip=71.218.115.0&id=o-ALaQ5w4MUHO0MjjdOwfz3LP7l4cflSgCnKJChDLl51i3&itag=133&aitags=133%2C134%2C135%2C160%2C242%2C243%2C244%2C278&source=youtube&requiressl=yes&mm=31%2C29&mn=sn-qxoedn7e%2Csn-qxo7rn7e&ms=au%2Crdu&mv=m&mvi=5&pl=15&initcwndbps=862500&mime=video%2Fmp4&gir=yes&clen=6997709&dur=228.795&lmt=1377574979076269&mt=1573782541&fvip=6&keepalive=yes&fexp=23842630&c=WEB&sparams=expire%2Cei%2Cip%2Cid%2Caitags%2Csource%2Crequiressl%2Cmime%2Cgir%2Cclen%2Cdur%2Clmt&lsparams=mm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AHylml4wRAIgJAz0pIscDE31pszTIzBEiOg7wzKnTnyhx6wNydutaHQCID-sFGFbsY9gFGrGk37s1OXS9qLsiRrwl5dnc9jH50CJ&sig=ALgxI2wwRgIhAPMILEVrJ7I3oSynQW56CrFi1H2SA_0j8R-w48_bYHleAiEAqjBQ8koCi0r5UiDwMdJAyya6Zh4NwBsaqsASN_oUP4Q=&ratebypass=yes', 'player_u

### Lets make a function that pulls a set number of frames
For now, we don't want every frame of the video
Lets lean towards more videos with a few frames over a lot of frames from a few videos

In [23]:
def get_frames(yt_id, output_dir, frames_wanted, format_wanted):
    yt_url = f"https://www.youtube.com/watch?v={yt_id}"
    ydl_options = {}
    
    ydl = youtube_dl.YoutubeDL(ydl_options)
    
    yt_info = ydl.extract_info(yt_url, download=False)
    
    formats = yt_info.get('formats', None)
    
    for f in formats:
        if f.get('format_note') == format_wanted:
            url = f.get('url', None)
            
            cap = cv2.VideoCapture(url)
            
            if not cap.isOpened():
                print('Video not opened')
                
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = int(cap.get(cv2.CAP_PROP_FPS))
            
            print(f"Total Frame Count: {total_frames}")
            print(f"FPS: {fps}")
            
            frame_iter = int(total_frames/(frames_wanted)+1)
            
            frame_num = 0
            
            while True:
                frame_num += frame_iter
                
                cap.set(1, frame_num-1)
                ret, frame = cap.read()
                
                if not ret:
                    break
                
                output_image = output_dir + f"/{yt_id}_{frame_num:06}.jpg"
                
                cv2.imwrite(output_image, frame)
                
                print(f"Writing Frame to: {output_image}")
            
            cap.release()
                
            break # Can have multiple of requested format, so get only 1

### Test function
Have to break after getting the asked for format_note because there can be multiples of each

In [24]:
get_frames('LLf2i_EK6RU', '../data/frames/test', 10, '240p')

[youtube] LLf2i_EK6RU: Downloading webpage
[youtube] LLf2i_EK6RU: Downloading video info webpage
Total Frame Count: 6857
FPS: 29
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_000686.jpg
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_001372.jpg
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_002058.jpg
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_002744.jpg
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_003430.jpg
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_004116.jpg
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_004802.jpg
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_005488.jpg
Writing Frame to: ../data/frames/test/LLf2i_EK6RU_006174.jpg
